In [8]:
# AES S-box table (256 entries)
S_BOX = [
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5,
    0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0,
    0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc,
    0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a,
    0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0,
    0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b,
    0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85,
    0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5,
    0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17,
    0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88,
    0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c,
    0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9,
    0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6,
    0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e,
    0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94,
    0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68,
    0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16
]

RCON = [0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x36]

def sub_word(word):
    return [S_BOX[b] for b in word]

def rot_word(word):
    return word[1:] + word[:1]

def key_expansion(key):
    Nk = 4
    Nr = 10
    total_words = 4 * (Nr + 1)
    
    w = []
    for i in range(Nk):
        word = key[4*i : 4*(i+1)]
        w.append(word)
        print(f"w[{i}] (initial key word): {' '.join(format(b, '02x') for b in word)}")

    for i in range(Nk, total_words):
        temp = w[i - 1].copy()
        print(f"\nGenerating w[{i}] from w[{i-1}]: {' '.join(format(b, '02x') for b in temp)}")

        if i % Nk == 0:
            temp = rot_word(temp)
            print(f" -> After RotWord: {' '.join(format(b, '02x') for b in temp)}")

            temp = sub_word(temp)
            print(f" -> After SubWord: {' '.join(format(b, '02x') for b in temp)}")

            temp[0] ^= RCON[i // Nk]
            print(f" -> After RCON XOR ({format(RCON[i // Nk], '02x')}): {' '.join(format(b, '02x') for b in temp)}")

        new_word = [w[i - Nk][j] ^ temp[j] for j in range(4)]
        w.append(new_word)
        print(f"w[{i}]: {' '.join(format(b, '02x') for b in new_word)}")

    return w

def print_round_keys(expanded_key):
    rounds = len(expanded_key) // 4
    expansion = ""
    for round_num in range(rounds):
        round_key = [byte for word in expanded_key[round_num*4:(round_num+1)*4] for byte in word]
        hex_key = ''.join('{:02x}'.format(b) for b in round_key)
        expansion += hex_key
        print(f"\nRound {round_num} key: {hex_key}")

    print("\nFinal key: ", expansion)

if __name__ == '__main__':
    key = bytes.fromhex('000102030405060708090a0b0c0d0e0f')
    key_bytes = list(key)
    
    expanded_key = key_expansion(key_bytes)
    print_round_keys(expanded_key)


w[0] (initial key word): 00 01 02 03
w[1] (initial key word): 04 05 06 07
w[2] (initial key word): 08 09 0a 0b
w[3] (initial key word): 0c 0d 0e 0f

Generating w[4] from w[3]: 0c 0d 0e 0f
 -> After RotWord: 0d 0e 0f 0c
 -> After SubWord: d7 ab 76 fe
 -> After RCON XOR (01): d6 ab 76 fe
w[4]: d6 aa 74 fd

Generating w[5] from w[4]: d6 aa 74 fd
w[5]: d2 af 72 fa

Generating w[6] from w[5]: d2 af 72 fa
w[6]: da a6 78 f1

Generating w[7] from w[6]: da a6 78 f1
w[7]: d6 ab 76 fe

Generating w[8] from w[7]: d6 ab 76 fe
 -> After RotWord: ab 76 fe d6
 -> After SubWord: 62 38 bb f6
 -> After RCON XOR (02): 60 38 bb f6
w[8]: b6 92 cf 0b

Generating w[9] from w[8]: b6 92 cf 0b
w[9]: 64 3d bd f1

Generating w[10] from w[9]: 64 3d bd f1
w[10]: be 9b c5 00

Generating w[11] from w[10]: be 9b c5 00
w[11]: 68 30 b3 fe

Generating w[12] from w[11]: 68 30 b3 fe
 -> After RotWord: 30 b3 fe 68
 -> After SubWord: 04 6d bb 45
 -> After RCON XOR (04): 00 6d bb 45
w[12]: b6 ff 74 4e

Generating w[13] from w[